In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

dir = '../../mnt/c/Users/louxsoen/Documents/P/Nerual-Network-SCA/Float-Point-NN-Mantissa/Trace/'

In [ ]:
relu = np.load(dir + 'relu.npy')
rmlp = np.load(dir + 'mlp.npy')
print(relu.shape)
print(rmlp.shape)

In [ ]:

cutsize = 400
strsize = 500
cutR = np.zeros([2, cutsize])
cutR[0] = relu[0][strsize:strsize + cutsize]
cutR[1] = relu[1][strsize:strsize + cutsize]

plt.figure(figsize=(10,2))
plt.plot(cutR[0])
plt.show()

print(np.corrcoef(cutR[0], cutR[1])[0,1])
if cutsize + strsize > 1050:    print("OVER SHOOT")

In [ ]:
endpt = 16000
corr_minus = np.zeros([255, endpt])
corr_plus  = np.zeros([255, endpt])

max_minus = np.zeros(endpt)
loc_minus = np.zeros(endpt)
max_plus  = np.zeros(endpt)
loc_plus  = np.zeros(endpt)
#for j in range(1, 2):

#  Minus ReLU
for a in range(0, 255):
    for i in range(0, endpt, 1):
        mcut = rmlp[a][i:i+cutsize]
        corr_minus[a][i] = np.corrcoef(mcut, cutR[0])[0,1]

#   Plus ReLU
for a in range(0, 255):
    for i in range(0, endpt, 1):
        mcut = rmlp[a][i:i+cutsize]
        corr_plus[a][i] = np.corrcoef(mcut, cutR[1])[0,1]


In [ ]:
plt.figure(figsize=(5,3))
plt.suptitle('corr_minus plot [cutted]')
plt.plot(corr_minus[8][13500:15500])
plt.show()
plt.figure(figsize=(5,1))
plt.suptitle('corr_minus plot [FULL]')
plt.plot(corr_minus[0])
plt.show()

plt.figure(figsize=(5,3))
plt.suptitle('corr_plus plot [cutted]')
plt.plot(corr_plus[8][13500:15500])
plt.show()
plt.figure(figsize=(5,1))
plt.suptitle('corr_plus plot [FULL]')
plt.plot(corr_plus[0])
plt.show()

In [ ]:
np.save(dir + "corr_minus", corr_minus)
np.save(dir + "corr_plus",  corr_plus)